In [16]:
import datetime
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import pandas as pd

def format_date(date):
    """
    Takes the date uniforms the format
    :param date_str:
        yyyymmdd(int)
    :return:
        list of ints: [yyyy,mm,dd]
    """
    return [int(str(date)[x:y]) for x, y in zip([0, 4, 6], [4, 6, 8])]

def get_date_range(range_list):
    """
    Takes a list of [start date, end date] and makes a list of the range (inclusive)
    :param range_list:
        [yyyymmdd, yyyymmdd]
    :return:
        a list of datetime.date objects
    """
    start_date = datetime.date(*format_date(range_list[0]))
    end_date = datetime.date(*format_date(range_list[1]))

    delta = end_date - start_date

    return [end_date - datetime.timedelta(days=n) for n in range(delta.days+1)]

In [17]:
def get_date(date):
    """
    Returns formatted date
    :param date:
        datetime.date instance
    """
    # convert date obj to strings and make sure single digit days have leading 0
    day, month, year = str(date.day).zfill(2), str(date.month), str(date.year)

    return 'mon={}&day={}&year={}'.format(month, day, year)

def get_url(date, site): 
    """
    creates the url to scrape from.
    :param date:
        formatted date to scrape from
        example: &mon=3&day=13&year=2016
    :param site:
        FantasyDuel or DraftKing
        example: fd | dk
    :return:
        formatted URL
    """
    return 'http://rotoguru1.com/cgi-bin/hyday.pl?game=' + site + '&' + get_date(date)


def fetch_player_data(date, url, site, data):
    """
    scrapes the player data from the given web page.
    :param date:
        current date to scrape
    :param url:
        url of the page 
        example: 'http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=3&day=13&year=2016'
    :param site:
        fd for FantasyDuel | dk for DraftKing
    :return:
        a list of player data
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # data.append(["Position", "Name", "Fantasy Points", "Salary", "Away", "Home", "Score", "Minutes", "Stats", 
    #             "Site", "ID", "Date", "First Name", "Last Name"])
    count = 0
    for n, player_row in enumerate(soup.find_all('tr')):
        pos = player_row.find('b')

        # Differentiate player rows by number of td tags.
        if len(player_row.findParents('table')) == 0 and\
                player_row.find('td', {'colspan':False}) and\
                len(player_row.find_all('b')) == 0:
            player_id = urlparse(player_row.find('a')['href']).query
            # Get the td tag text and format it.
            player_data = [data.text.replace(u'\xa0', u'').strip() for data in player_row.find_all('td')]
            player_data.append(site)
            player_data.append(player_id)
            player_data.append(date)
            if len(player_data[1]) > 0: 
                player_data.append(player_data[1].split(",")[1].replace("^", ""))
                player_data.append(player_data[1].split(",")[0])
                if (player_data[7] != "NA" and player_data[7] != "DNP"): 
                    data.append(player_data)
                count += 1
    print(count, "rows of player data scraped \n")

In [18]:
def main(date_range, site):
    # print(str(date_range[0]) + "-" +str(date_range[1]) + '.csv')
    date_list = get_date_range(date_range)
    data = []
    data.append(["Position", "Name", "Fantasy Points", "Salary", "Away", "Home", "Score", "Minutes", "Stats", 
                "Site", "ID", "Date", "First Name", "Last Name"])
    for date in date_list:
        print(date)
        
        url = get_url(date, site)
        print(url)

        fetch_player_data(str(date), url, site, data)
    
    df = pd.DataFrame(data)
    df.to_csv('results/rotoGuru/' + str(date_range[0]) + '-' + str(date_range[1]) + '-' + site 
              + '.csv', index=False, header=False)
    
    print("SUCCESS \n")    

In [20]:
# NBA SEASON 2015-16
# Date: 2015-10-27 - 2016-06-19
main([20151027, 20160619], 'fd')
main([20151027, 20160619], 'dk')

2016-06-19
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=19&year=2016
18 rows of player data scraped 

2016-06-18
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=18&year=2016
0 rows of player data scraped 

2016-06-17
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=17&year=2016
0 rows of player data scraped 

2016-06-16
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=16&year=2016
24 rows of player data scraped 

2016-06-15
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=15&year=2016
0 rows of player data scraped 

2016-06-14
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=14&year=2016
0 rows of player data scraped 

2016-06-13
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=13&year=2016
21 rows of player data scraped 

2016-06-12
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=12&year=2016
0 rows of player data scraped 

2016-06-11
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=11&year=2016
0 rows of player data

280 rows of player data scraped 

2016-04-07
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=07&year=2016
141 rows of player data scraped 

2016-04-06
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=06&year=2016
227 rows of player data scraped 

2016-04-05
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=05&year=2016
307 rows of player data scraped 

2016-04-04
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=04&year=2016
0 rows of player data scraped 

2016-04-03
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=03&year=2016
309 rows of player data scraped 

2016-04-02
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=02&year=2016
139 rows of player data scraped 

2016-04-01
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=01&year=2016
281 rows of player data scraped 

2016-03-31
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=3&day=31&year=2016
167 rows of player data scraped 

2016-03-30
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd

231 rows of player data scraped 

2016-01-25
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=25&year=2016
260 rows of player data scraped 

2016-01-24
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=24&year=2016
115 rows of player data scraped 

2016-01-23
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=23&year=2016
225 rows of player data scraped 

2016-01-22
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=22&year=2016
256 rows of player data scraped 

2016-01-21
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=21&year=2016
144 rows of player data scraped 

2016-01-20
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=20&year=2016
319 rows of player data scraped 

2016-01-19
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=19&year=2016
112 rows of player data scraped 

2016-01-18
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=18&year=2016
285 rows of player data scraped 

2016-01-17
http://rotoguru1.com/cgi-bin/hyday.pl?game=

203 rows of player data scraped 

2015-11-14
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=14&year=2015
195 rows of player data scraped 

2015-11-13
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=13&year=2015
318 rows of player data scraped 

2015-11-12
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=12&year=2015
85 rows of player data scraped 

2015-11-11
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=11&year=2015
310 rows of player data scraped 

2015-11-10
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=10&year=2015
203 rows of player data scraped 

2015-11-09
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=09&year=2015
205 rows of player data scraped 

2015-11-08
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=08&year=2015
145 rows of player data scraped 

2015-11-07
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=07&year=2015
256 rows of player data scraped 

2015-11-06
http://rotoguru1.com/cgi-bin/hyday.p

90 rows of player data scraped 

2016-04-26
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=26&year=2016
60 rows of player data scraped 

2016-04-25
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=25&year=2016
89 rows of player data scraped 

2016-04-24
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=24&year=2016
120 rows of player data scraped 

2016-04-23
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=23&year=2016
120 rows of player data scraped 

2016-04-22
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=22&year=2016
88 rows of player data scraped 

2016-04-21
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=21&year=2016
90 rows of player data scraped 

2016-04-20
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=20&year=2016
90 rows of player data scraped 

2016-04-19
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=19&year=2016
61 rows of player data scraped 

2016-04-18
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=

0 rows of player data scraped 

2016-02-13
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=13&year=2016
0 rows of player data scraped 

2016-02-12
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=12&year=2016
0 rows of player data scraped 

2016-02-11
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=11&year=2016
60 rows of player data scraped 

2016-02-10
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=10&year=2016
357 rows of player data scraped 

2016-02-09
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=09&year=2016
148 rows of player data scraped 

2016-02-08
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=08&year=2016
297 rows of player data scraped 

2016-02-07
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=07&year=2016
119 rows of player data scraped 

2016-02-06
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=06&year=2016
298 rows of player data scraped 

2016-02-05
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=

177 rows of player data scraped 

2015-12-03
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=12&day=03&year=2015
179 rows of player data scraped 

2015-12-02
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=12&day=02&year=2015
265 rows of player data scraped 

2015-12-01
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=12&day=01&year=2015
176 rows of player data scraped 

2015-11-30
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=11&day=30&year=2015
237 rows of player data scraped 

2015-11-29
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=11&day=29&year=2015
235 rows of player data scraped 

2015-11-28
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=11&day=28&year=2015
208 rows of player data scraped 

2015-11-27
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=11&day=27&year=2015
346 rows of player data scraped 

2015-11-26
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=11&day=26&year=2015
0 rows of player data scraped 

2015-11-25
http://rotoguru1.com/cgi-bin/hyday.pl

In [22]:
# NBA SEASON 2016-17
# Date: 2016-10-25 - 2017-06-18
main([20161025, 20170618], 'fd')
main([20161025, 20170618], 'dk')

2017-06-18
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=18&year=2017
0 rows of player data scraped 

2017-06-17
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=17&year=2017
0 rows of player data scraped 

2017-06-16
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=16&year=2017
0 rows of player data scraped 

2017-06-15
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=15&year=2017
0 rows of player data scraped 

2017-06-14
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=14&year=2017
0 rows of player data scraped 

2017-06-13
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=13&year=2017
0 rows of player data scraped 

2017-06-12
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=12&year=2017
18 rows of player data scraped 

2017-06-11
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=11&year=2017
0 rows of player data scraped 

2017-06-10
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=10&year=2017
0 rows of player data s

253 rows of player data scraped 

2017-04-06
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=06&year=2017
173 rows of player data scraped 

2017-04-05
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=05&year=2017
228 rows of player data scraped 

2017-04-04
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=04&year=2017
310 rows of player data scraped 

2017-04-03
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=03&year=2017
27 rows of player data scraped 

2017-04-02
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=02&year=2017
341 rows of player data scraped 

2017-04-01
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=01&year=2017
141 rows of player data scraped 

2017-03-31
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=3&day=31&year=2017
307 rows of player data scraped 

2017-03-30
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=3&day=30&year=2017
144 rows of player data scraped 

2017-03-29
http://rotoguru1.com/cgi-bin/hyday.pl?game=f

171 rows of player data scraped 

2017-01-23
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=23&year=2017
263 rows of player data scraped 

2017-01-22
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=22&year=2017
115 rows of player data scraped 

2017-01-21
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=21&year=2017
321 rows of player data scraped 

2017-01-20
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=20&year=2017
263 rows of player data scraped 

2017-01-19
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=19&year=2017
141 rows of player data scraped 

2017-01-18
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=18&year=2017
262 rows of player data scraped 

2017-01-17
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=17&year=2017
141 rows of player data scraped 

2017-01-16
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=16&year=2017
260 rows of player data scraped 

2017-01-15
http://rotoguru1.com/cgi-bin/hyday.pl?game=

146 rows of player data scraped 

2016-11-12
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=12&year=2016
317 rows of player data scraped 

2016-11-11
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=11&year=2016
231 rows of player data scraped 

2016-11-10
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=10&year=2016
116 rows of player data scraped 

2016-11-09
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=09&year=2016
319 rows of player data scraped 

2016-11-08
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=08&year=2016
175 rows of player data scraped 

2016-11-07
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=07&year=2016
200 rows of player data scraped 

2016-11-06
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=06&year=2016
178 rows of player data scraped 

2016-11-05
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=05&year=2016
232 rows of player data scraped 

2016-11-04
http://rotoguru1.com/cgi-bin/hyday.

60 rows of player data scraped 

2017-04-25
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=25&year=2017
87 rows of player data scraped 

2017-04-24
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=24&year=2017
90 rows of player data scraped 

2017-04-23
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=23&year=2017
119 rows of player data scraped 

2017-04-22
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=22&year=2017
120 rows of player data scraped 

2017-04-21
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=21&year=2017
89 rows of player data scraped 

2017-04-20
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=20&year=2017
90 rows of player data scraped 

2017-04-19
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=19&year=2017
89 rows of player data scraped 

2017-04-18
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=18&year=2017
90 rows of player data scraped 

2017-04-17
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=

119 rows of player data scraped 

2017-02-11
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=11&year=2017
237 rows of player data scraped 

2017-02-10
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=10&year=2017
269 rows of player data scraped 

2017-02-09
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=09&year=2017
147 rows of player data scraped 

2017-02-08
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=08&year=2017
355 rows of player data scraped 

2017-02-07
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=07&year=2017
89 rows of player data scraped 

2017-02-06
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=06&year=2017
326 rows of player data scraped 

2017-02-05
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=05&year=2017
90 rows of player data scraped 

2017-02-04
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=04&year=2017
293 rows of player data scraped 

2017-02-03
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk

269 rows of player data scraped 

2016-12-01
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=12&day=01&year=2016
180 rows of player data scraped 

2016-11-30
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=11&day=30&year=2016
268 rows of player data scraped 

2016-11-29
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=11&day=29&year=2016
178 rows of player data scraped 

2016-11-28
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=11&day=28&year=2016
210 rows of player data scraped 

2016-11-27
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=11&day=27&year=2016
240 rows of player data scraped 

2016-11-26
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=11&day=26&year=2016
149 rows of player data scraped 

2016-11-25
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=11&day=25&year=2016
450 rows of player data scraped 

2016-11-24
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=11&day=24&year=2016
0 rows of player data scraped 

2016-11-23
http://rotoguru1.com/cgi-bin/hyday.pl

In [25]:
# NBA SEASON 2017-18
# Date: 2017-10-17 - 2018-06-17
main([20171017, 20180617], 'fd')
main([20171017, 20180617], 'dk')

2018-06-17
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=17&year=2018
0 rows of player data scraped 

2018-06-16
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=16&year=2018
0 rows of player data scraped 

2018-06-15
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=15&year=2018
0 rows of player data scraped 

2018-06-14
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=14&year=2018
0 rows of player data scraped 

2018-06-13
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=13&year=2018
0 rows of player data scraped 

2018-06-12
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=12&year=2018
0 rows of player data scraped 

2018-06-11
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=11&year=2018
0 rows of player data scraped 

2018-06-10
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=10&year=2018
0 rows of player data scraped 

2018-06-09
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=6&day=09&year=2018
0 rows of player data sc

312 rows of player data scraped 

2018-04-05
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=05&year=2018
192 rows of player data scraped 

2018-04-04
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=04&year=2018
191 rows of player data scraped 

2018-04-03
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=03&year=2018
413 rows of player data scraped 

2018-04-02
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=02&year=2018
0 rows of player data scraped 

2018-04-01
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=4&day=01&year=2018
403 rows of player data scraped 

2018-03-31
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=3&day=31&year=2018
158 rows of player data scraped 

2018-03-30
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=3&day=30&year=2018
284 rows of player data scraped 

2018-03-29
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=3&day=29&year=2018
162 rows of player data scraped 

2018-03-28
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd

293 rows of player data scraped 

2018-01-23
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=23&year=2018
163 rows of player data scraped 

2018-01-22
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=22&year=2018
291 rows of player data scraped 

2018-01-21
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=21&year=2018
128 rows of player data scraped 

2018-01-20
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=20&year=2018
291 rows of player data scraped 

2018-01-19
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=19&year=2018
226 rows of player data scraped 

2018-01-18
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=18&year=2018
124 rows of player data scraped 

2018-01-17
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=17&year=2018
323 rows of player data scraped 

2018-01-16
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=1&day=16&year=2018
132 rows of player data scraped 

2018-01-15
http://rotoguru1.com/cgi-bin/hyday.pl?game=

292 rows of player data scraped 

2017-11-12
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=12&year=2017
129 rows of player data scraped 

2017-11-11
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=11&year=2017
360 rows of player data scraped 

2017-11-10
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=10&year=2017
258 rows of player data scraped 

2017-11-09
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=09&year=2017
163 rows of player data scraped 

2017-11-08
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=08&year=2017
163 rows of player data scraped 

2017-11-07
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=07&year=2017
326 rows of player data scraped 

2017-11-06
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=06&year=2017
96 rows of player data scraped 

2017-11-05
http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=11&day=05&year=2017
323 rows of player data scraped 

2017-11-04
http://rotoguru1.com/cgi-bin/hyday.p

60 rows of player data scraped 

2018-05-02
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=5&day=02&year=2018
19 rows of player data scraped 

2018-05-01
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=5&day=01&year=2018
60 rows of player data scraped 

2018-04-30
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=30&year=2018
17 rows of player data scraped 

2018-04-29
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=29&year=2018
60 rows of player data scraped 

2018-04-28
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=28&year=2018
60 rows of player data scraped 

2018-04-27
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=27&year=2018
90 rows of player data scraped 

2018-04-26
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=26&year=2018
20 rows of player data scraped 

2018-04-25
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4&day=25&year=2018
120 rows of player data scraped 

2018-04-24
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=4

24 rows of player data scraped 

2018-02-17
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=17&year=2018
0 rows of player data scraped 

2018-02-16
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=16&year=2018
20 rows of player data scraped 

2018-02-15
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=15&year=2018
66 rows of player data scraped 

2018-02-14
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=14&year=2018
387 rows of player data scraped 

2018-02-13
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=13&year=2018
193 rows of player data scraped 

2018-02-12
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=12&year=2018
195 rows of player data scraped 

2018-02-11
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=11&year=2018
249 rows of player data scraped 

2018-02-10
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=2&day=10&year=2018
214 rows of player data scraped 

2018-02-09
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mo

266 rows of player data scraped 

2017-12-07
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=12&day=07&year=2017
131 rows of player data scraped 

2017-12-06
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=12&day=06&year=2017
332 rows of player data scraped 

2017-12-05
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=12&day=05&year=2017
98 rows of player data scraped 

2017-12-04
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=12&day=04&year=2017
366 rows of player data scraped 

2017-12-03
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=12&day=03&year=2017
149 rows of player data scraped 

2017-12-02
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=12&day=02&year=2017
268 rows of player data scraped 

2017-12-01
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=12&day=01&year=2017
263 rows of player data scraped 

2017-11-30
http://rotoguru1.com/cgi-bin/hyday.pl?game=dk&mon=11&day=30&year=2017
167 rows of player data scraped 

2017-11-29
http://rotoguru1.com/cgi-bin/hyday.p